In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from statistics import mean
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
pd.set_option('display.float_format', '{:.1f}'.format)

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
%cd /Users/macbookpro/Documents/COMP90042_NLP/Project/COMP90042_2024_Project

/Users/macbookpro/Documents/COMP90042_NLP/Project/COMP90042_2024_Project


In [8]:
with open("data/train-claims.json", 'r') as file:
    train_claims = json.load(file)

with open("data/dev-claims.json", 'r') as file:
    dev_claims = json.load(file)

with open("data/test-claims-unlabelled.json", 'r') as file:
    test_claims_unlabelled = json.load(file)

with open("data/dev_pred.json", 'r') as file:
    dev_pred = json.load(file)

with open("data/evidence.json", 'r') as file:
    evidence = json.load(file)

with open("data/test_pred.json", 'r') as file:
    test_claims_pred = json.load(file)

In [9]:
train_claims_dic = [{"claim_id": key,
                    "claim_text": value["claim_text"],
                    "claim_label": value["claim_label"],
                    "evidences_id": value["evidences"]} for (key,value) in train_claims.items()]
train_claims_df = pd.json_normalize(train_claims_dic)

dev_claims_dic = [{"claim_id": key,
                    "claim_text": value["claim_text"],
                    "claim_label": value["claim_label"],
                    "evidences_id": value["evidences"]} for (key,value) in dev_claims.items()]
dev_claims_df = pd.json_normalize(dev_claims_dic)

test_claims_dic = [{"claim_id": key,
                    "claim_text": value["claim_text"]} for (key,value) in test_claims_unlabelled.items()]
test_claims_df = pd.json_normalize(test_claims_dic)

evidence_dic = [{"evidence_id": key,
                             "evidence_text": value
                             } for (key, value) in evidence.items()]
evidence_df = pd.json_normalize(evidence_dic)

In [ ]:
train_df = pd.DataFrame(columns=['claim_id','text'])
train_y = []
for key, value in train_claims.items():
    row_to_add = pd.DataFrame({
        "claim_id" :[key],
        "text": ["<sep>".join(["<cls>" +value['claim_text']] + [evidence[evd_id] for evd_id in value['evidences']])]
    })
    train_df = train_df.append(row_to_add, ignore_index = True)


In [ ]:
label_list = ["SUPPORTS", "REFUTES", "NOT_ENOUGH_INFO", "DISPUTED"]
label_map = {"SUPPORTS":0, "REFUTES":1, "NOT_ENOUGH_INFO":2, "DISPUTED":3}

In [ ]:
def reconstruct_data(claims,evidence):
    X = pd.DataFrame(columns=['claim_id','text','num_evidences','evidence_ids'])
    Y = []
    # prepare text: <cls> claim text <sep> evidence_1 text <sep> evidence_2 text
    for key, value in claims.items():
        row_to_add = pd.DataFrame({
            "claim_id" :[key],
            "text": ["<sep>".join(["<cls>" +value['claim_text']] + [evidence[evd_id] for evd_id in value['evidences']])],
            'num_evidences': len(value['evidences']),
            'evidence_ids': [value['evidences']]
        })
        Y.append(value['claim_label'])
        X = X.append(row_to_add, ignore_index = True)
    X_list = X['text'].values
    Y = list(map(lambda x: label_map[x], Y))
    return X, X_list, Y

In [ ]:
def reconstruct_X_only(claims, evidence):
    X = pd.DataFrame(columns=['claim_id','text','num_evidences','evidence_ids'])

    # prepare text: <cls> claim text <sep> evidence_1 text <sep> evidence_2 text
    for key, value in claims.items():
        row_to_add = pd.DataFrame({
            "claim_id" :[key],
            "text": ["<sep>".join(["<cls>" +value['claim_text']] + [evidence[evd_id] for evd_id in value['evidences']])],
            'num_evidences': len(value['evidences']),
            'evidence_ids': [value['evidences']]
        })
        X = X.append(row_to_add, ignore_index = True)
    X_list = X['text'].values
    
    return X, X_list

In [ ]:
train_df, train_X, train_y = reconstruct_data(train_claims,evidence)
dev_df_true, dev_X_true, dev_y_true = reconstruct_data(dev_claims,evidence)

In [ ]:
dev_df_pred, dev_X_pred = reconstruct_X_only(dev_pred,evidence)
test_df, test_X = reconstruct_X_only(test_claims_pred,evidence)

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from sklearn.feature_extraction.text import CountVectorizer
import torch